In [1]:
import sys
sys.path.insert(0, '../')
import myTA
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np

from strategies import Backtesting, MACDZeroHistPrediction, makeAnalysis, SharpeRatio, get_df, init_backtesting_results_file, get_allowedIntervals

c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
sharpe_ratio = SharpeRatio()
allowedIntervals = get_allowedIntervals()
init_backtesting_results_file("backtest_resluts/backtesting_results.csv")

In [3]:
tickers = ['uber']

linerPredictionWindows = [4]
fastperiods = [12]
slowperiods = [26]
signalperiods = [9]

# linerPredictionWindows = np.arange(2,6)
# fastperiods = np.arange(8,18)
# slowperiods = np.arange(20,30)
# signalperiods = np.arange(5,13)

df = pd.DataFrame()

for i in range(len(allowedIntervals)):
    end = dt.datetime.now()
    start = end - allowedIntervals.iloc[i]['back_allowed']
    interval = allowedIntervals.iloc[i]['intervals']

    for ticker in tickers:
        data = yf.download(ticker, start=start, end=end, interval=interval, progress=False)

        for linerPredictionWindow in linerPredictionWindows:
            for fastperiod in fastperiods:
                for slowperiod in slowperiods:
                    for signalperiod in signalperiods:
                        strategy = MACDZeroHistPrediction(linerPredictionWindow, ticker, start, end, interval, fastperiod, slowperiod, signalperiod)
                        data = makeAnalysis(strategy, data)
                        strategy.getBackTestSignals(data)

                        bt = Backtesting(100000, strategy)
                        bt.calculateProfit()
                        df = get_df(df, bt, sharpe_ratio)

    df.to_csv('backtest_resluts/backtesting_results.csv', mode='a', index=False, header=False)
    df = pd.DataFrame()

In [4]:
results = pd.read_csv("backtest_resluts/backtesting_results.csv")
results

,interval,start,end,ticker,linerPredictionWindow,fastperiod,slowperiod,signalperiod,budget_start,result,sharpe_ratio,ROI
0,1m,2023-02-24 20:23:59.879882,2023-03-01 20:23:59.879882,uber,4,12,26,9,100000,-0.0,-125.605944,-0.000
1,2m,2023-01-01 20:24:00.374023,2023-03-01 20:24:00.374023,uber,4,12,26,9,100000,11.0,-13.410396,0.011
2,5m,2023-01-01 20:24:02.383475,2023-03-01 20:24:02.383475,uber,4,12,26,9,100000,31.0,-14.871948,0.031
3,15m,2023-01-01 20:24:03.409009,2023-03-01 20:24:03.409009,uber,4,12,26,9,100000,25.0,-34.933495,0.025
4,30m,2023-01-01 20:24:03.924206,2023-03-01 20:24:03.924206,uber,4,12,26,9,100000,9.0,-53.395976,0.009
5,1h,2021-03-02 20:24:04.394898,2023-03-01 20:24:04.394898,uber,4,12,26,9,100000,-14.0,-12.656171,-0.014
6,90m,2023-01-01 20:24:05.712379,2023-03-01 20:24:05.712379,uber,4,12,26,9,100000,-0.0,-90.179355,-0.000
7,1d,1971-03-14 20:24:05.963380,2023-03-01 20:24:05.963380,uber,4,12,26,9,100000,-6.0,-6.757956,-0.006
